Подготовка датасета начинается с обрезки. 
Все файлы из папки targetsPath обрезаются и переносятся в папку pathToAudio
При этом они сортируются. Слишком длинные или короткие отсекаются.

In [ ]:
import os
from workWithAudio import *

targetsPath = 'pandModel\\data\\train'
trimmedPath = 'pandModel\\data\\trimmedNice\\train'
files = os.listdir(targetsPath)
i = 0
for file in files:
    i+=1
    if(i%100 == 0):
        print(fr"Обработано {i} аудиозаписей")

    newAudio = workWithAudio(targetsPath, file, trimmedPath=trimmedPath)
    newAudio.cropAndSort()

Пропишем пути:
Файл-каталог с [название_аудиофайла     пол] 
Путь к папке с аудиозаписями
Для готовых датасетов
Для готовых моделей
Переменная frameLength определяет длину тактов. Если эта величина будет меньше 20, обучение пойдет куда быстрее, но отнимет несколько процентов от точности. Если больше 500, прорешиваться будет долго.

In [1]:
pathToFile = "D:\\SD\\voiceGenderClassifier\\data\\targets.tsv"
# pathToAudio = "D:\\SD\\voiceGenderClassifier\\data\\trimmedAudio\\train"
pathToAudio = "D:\\SD\\voiceGenderClassifier\\data\\train"
pathToDataset = "D:\\SD\\voiceGenderClassifier\\dataset"
pathToDataset = "D:\\SD\\voiceGenderClassifier\\dataset_good"
pathToModel = "D:\\SD\\voiceGenderClassifier\\models"

frameLength = 200

Подготавливаем датасет на основе содержимого папки

In [3]:
from makeDataset import *

maked = MakeDataset(pathToAudio, pathToFile, pathToDataset)
df = maked.cutAll(frameLength)
df.head()

AttributeError: 'list' object has no attribute 'shape'

...Или просто получаем его из файла датасета

In [7]:
import pandas as pd
import numpy as np
# df = pd.read_csv(pathToDataset+"\\dataset_60_9656.scv")
df = pd.read_csv(pathToDataset+'\\dataset_200_100000.scv')

Нарезаем датасет на переменные. test_size и random_state можно менять, что даст ощутимый прирост точности.
Впрочем, вероятно он довольно рандомный и связан с не совсем чистой обрезкой массива обучения.

In [1]:
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_x = scaler.fit_transform(df.drop(columns='sex'))

X_train, X_test, y_train, y_test = train_test_split(
        scaled_x, df.sex, test_size=0.1, random_state=705)


NameError: name 'df' is not defined

ExtraTreesClassifier показал себя лучше других алгоритмов в данном случае.
Параметр n_estimators можно понизить до 100, если нам не нужны полпроцента.
Текущая максимальная точность: 0.9788
Параметры:
frameLength = 200
test_size=0.1   // Это не менять, как и файл датасета
random_state=705 // Иначе потеряем процент. Или это не важно?
ExtraTreesClassifier(n_estimators=500)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

etc = ExtraTreesClassifier(n_estimators=30)
etc.fit(X_train, y_train)
score = etc.score(X_test, y_test)
print(score)

model = etc

Сохраняем модель, включая в имя информацию об алгоритме, длине такта и точности до 4 знака

In [10]:
from pickle import dump
import os.path

scTxt = str(int(score*10000))
pathToFile = pathToModel+fr'//model_new_{model.__class__.__name__}_{frameLength}_{scTxt}.sav'
if(not os.path.isfile(pathToFile)):
    open(pathToFile, 'x')
dump(model, open(pathToFile, 'wb'))